In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv('D:/datasets/train.csv')
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [3]:
data.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

In [4]:
data.shape

(891, 8)

In [5]:
data.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [6]:
from sklearn.model_selection import train_test_split
xtrain, xtest,ytrain,ytest = train_test_split(data.drop('Survived', axis=1), data['Survived'], test_size=0.1)
xtrain.shape

(801, 7)

In [7]:
data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [9]:
#age column data missng in percentage
age_datamissing=data['Age'].isnull().sum()/(data.shape[0] / 100)
age_datamissing

19.865319865319865

In [10]:
embarked_datamissing = data['Embarked'].isnull().sum()/(data.shape[0]/100)
embarked_datamissing

0.2244668911335578

In [11]:
#handling the missing values of Age and Embarked columns
si_age = SimpleImputer(strategy='mean')
si_embarked = SimpleImputer(strategy='most_frequent')

train_age_transformed=si_age.fit_transform(xtrain[['Age']])
train_embarked_transformed=si_embarked.fit_transform(xtrain[['Embarked']])

test_age_transformed=si_age.transform(xtest[['Age']])
test_embarked_transformed=si_embarked.transform(xtest[['Embarked']])

In [12]:
test_age_transformed[5:11]

array([[27.        ],
       [29.74675507],
       [28.        ],
       [24.        ],
       [16.        ],
       [29.74675507]])

In [13]:
data['Sex'].nunique()

2

ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')<br>ohe_embarked = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore') <br>
NOTE: while using OneHotEncoder, we can not use both drop='first' and handle_unknown='ignore'. Example if we had three features
a,b,c if we encode all the features it would be [1 0 0 ], [0 1 0] anf [0 0 1] respectively. drop='False' would drop the column
a which means a will be representated by [0 0] later on. Similar to drop='False', handle_unknown='ignore' would also make the
new feature's vector to all zeros. That means feature a's vector values and newly introduced feature's vector value will be all
zeros which is definitely create a confusion... That's why sklean doesnot allow both the parameters to be used at a single time.
<br> <br>
The reasons why we should not use drop='first' <br>
https://www.youtube.com/watch?v=NYtwyvyvDEk <br>
https://inmachineswetrust.com/posts/drop-first-columns/

In [23]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False, handle_unknown='ignore')


ohe.fit(xtrain[['Sex']])
ohe_embarked.fit(train_embarked_transformed)

train_sex = ohe.transform(xtrain[['Sex']])
test_sex =  ohe.transform(xtest[['Sex']])

train_embarked = ohe_embarked.transform(train_embarked_transformed)
test_embarked = ohe_embarked.transform(test_embarked_transformed)

train_sex[0:4]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [29]:
train_embarked[21:27]

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

# using pipeline to make all the work efficient.

In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [42]:
xtrain.sample(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
803,3,male,0.42,0,1,8.5167,C
247,2,female,24.00,0,2,14.5000,S
53,2,female,29.00,1,0,26.0000,S


In [56]:
ctf1 = ColumnTransformer(transformers=[
    ('impute_age', SimpleImputer(strategy='mean'),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
], remainder='passthrough')

ctf2 = ColumnTransformer(transformers=[
    ('ohe_Sex', OneHotEncoder(handle_unknown='ignore', sparse=False),[1,6])
],remainder='passthrough')

# ctf3 = ColumnTransformer(transformers=[
#     ('standardscaler',StandardScaler(), slice(0,10))
# ])

In [57]:
pipe= Pipeline(
[
    ('1',ctf1),
    ('2',ctf2),
#     ('3',ctf3)
]
)

In [58]:
pipe.fit(xtrain)

Pipeline(steps=[('1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_Sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])]))])

In [59]:
t = pipe.transform(xtrain)
t

array([[0.0, 0.0, 1.0, ..., 'male', 0, 0],
       [0.0, 0.0, 1.0, ..., 'female', 4, 2],
       [0.0, 0.0, 1.0, ..., 'male', 0, 0],
       ...,
       [0.0, 0.0, 1.0, ..., 'male', 0, 0],
       [0.0, 0.0, 1.0, ..., 'male', 0, 0],
       [0.0, 0.0, 1.0, ..., 'male', 0, 0]], dtype=object)